# Cat Chat Bot Expert

In [ ]:
# Installation command
%pip install -r requirements.txt


In [ ]:
#%pip install pytube
#%pip install --upgrade pytube
#%pip install yt-dlp
#%pip install moviepy
#%pip install whisper
#%pip install chromadb sentence-transformers
#%pip install git+https://github.com/openai/whisper.git
#%pip install pytubefix
#%pip install chromadb
#%pip install langchain
#%pip install openai
#%pip install opencv-python
#%pip install langchain_openai
#%pip install --upgrade huggingface_hub
#%pip install --upgrade sentence-transformers




# Load the key

In [ ]:

from dotenv import load_dotenv
import os
import langsmith

# Specify the path to the .env file
dotenv_path = "./notebooks/apikey.env" #Change if your env is in a diffretn folder
load_dotenv(dotenv_path)

# Ensure required environment variables are loaded
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

# Check if all environment variables are set; raise an error if any are missing
if not all([OPENAI_API_KEY, LANGCHAIN_API_KEY, HUGGINGFACEHUB_API_TOKEN]):
    raise ValueError("Some required API keys are missing in the .env file.")

# Enable LangSmith tracing with environment variables
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "cat_expert_knowledge"

# Initialize LangSmith Client
from langsmith import Client
client = Client(api_key=LANGCHAIN_API_KEY)


Load the videos to learn.
Video to Audio

In [ ]:
from pytubefix import YouTube
import os
import hashlib

# Define the directory for downloaded audio files
output_path = "path/to/m4a"
os.makedirs(output_path, exist_ok=True)  # Ensure the directory exists

# Replace with your list of YouTube URLs
video_urls = ["https://youtu.be/ZUcVUFvmDFE?si=z9GfOAWF1qothiKs", 
            "https://youtu.be/4DlJYcfiRu4?si=cUVT9L5dEEdkcSt_", 
            "https://youtu.be/rxInrRQLEmM?si=Ai7wHN0dI--cns0x", 
            "https://youtu.be/gxlNfh5ukMw?si=naO3n4VZeXx3PlOs", 
            "https://youtu.be/ojS7XwtoXtw?si=NpNSef7dCm_LnFPv", 
            "https://youtu.be/tsYT7yIOdqQ?si=hdGEpxlmFNMf7NNQ", 
            "https://youtu.be/tsYT7yIOdqQ?si=e_Zdh2dGpqempHR8", 
            "https://youtu.be/UWohxDOXsl4?si=y1nXlUZYw6uzkc8n", 
            "https://youtu.be/gZrwcoiy_gY?si=ksfYE03t6xtuxUL0", 
            "https://youtu.be/lSDI5diNu4Y?si=Q-In6zMD4ZpuaPIz", 
            "https://youtu.be/8aCGL9GpVUg?si=_0yF1U1thjwJqyPY", 
            "https://youtu.be/VjOXvD7OvrE?si=t6xugNxLeMjpsi7E", 
            "https://youtu.be/FzifwTnCV5s?si=sR_u4kG-4NoQx5Ux", 
            "https://youtu.be/XreeFU7RYeI?si=hsc9WO24dJP6AfV2", 
            "https://youtu.be/-4O97jw_8Bc?si=pC14dgZ_f4mXdYPv" ]

# Function to create a unique filename from URL
def generate_filename(url, extension="m4a"):
    """Creates a unique filename for each URL based on its hash."""
    return hashlib.md5(url.encode()).hexdigest() + f".{extension}"

# Download each video as audio and handle errors
failed_downloads = []  # To log any failed downloads

for url in video_urls:
    # Generate filename and check if it exists
    filename = generate_filename(url)
    file_path = os.path.join(output_path, filename)
    
    if os.path.exists(file_path):
        print(f"Already downloaded: {url}")
        continue  # Skip downloading if file exists
    
    # Download video if not already downloaded
    try:
        yt = YouTube(url)
        video = yt.streams.filter(only_audio=True).first()
        video.download(output_path=output_path, filename=filename)
        print(f"Downloaded: {url}")
    except Exception as e:
        print(f"Failed to download {url}: {e}")
        failed_downloads.append(url)

# Optional: log failed downloads if any
if failed_downloads:
    print("Failed Downloads:", failed_downloads)
    # You could write these to a log file for later review

Already downloaded: https://youtu.be/ZUcVUFvmDFE?si=z9GfOAWF1qothiKs
Already downloaded: https://youtu.be/4DlJYcfiRu4?si=cUVT9L5dEEdkcSt_
Already downloaded: https://youtu.be/rxInrRQLEmM?si=Ai7wHN0dI--cns0x
Already downloaded: https://youtu.be/gxlNfh5ukMw?si=naO3n4VZeXx3PlOs
Already downloaded: https://youtu.be/ojS7XwtoXtw?si=NpNSef7dCm_LnFPv
Already downloaded: https://youtu.be/tsYT7yIOdqQ?si=hdGEpxlmFNMf7NNQ
Already downloaded: https://youtu.be/tsYT7yIOdqQ?si=e_Zdh2dGpqempHR8
Already downloaded: https://youtu.be/UWohxDOXsl4?si=y1nXlUZYw6uzkc8n
Already downloaded: https://youtu.be/gZrwcoiy_gY?si=ksfYE03t6xtuxUL0
Already downloaded: https://youtu.be/lSDI5diNu4Y?si=Q-In6zMD4ZpuaPIz
Already downloaded: https://youtu.be/8aCGL9GpVUg?si=_0yF1U1thjwJqyPY
Already downloaded: https://youtu.be/VjOXvD7OvrE?si=t6xugNxLeMjpsi7E
Already downloaded: https://youtu.be/FzifwTnCV5s?si=sR_u4kG-4NoQx5Ux
Already downloaded: https://youtu.be/XreeFU7RYeI?si=hsc9WO24dJP6AfV2
Already downloaded: https://youtu.

# Transcriptions done by Whisper

In [ ]:
import whisper
import os

# Load Whisper model
model = whisper.load_model("base")

# Directory containing your .m4a files
audio_dir = "/notebooks/path/to/m4a/"
transcription_dir = "/notebooks/path/to/transcriptions/"

# Ensure the transcription directory exists
os.makedirs(transcription_dir, exist_ok=True)

# List all .m4a files in the audio directory
audio_files = [f for f in os.listdir(audio_dir) if f.endswith(".m4a")]

# Transcribe each audio file and save it as a .txt file
for audio_file in audio_files:
    audio_path = os.path.join(audio_dir, audio_file)
    try:
        transcription = model.transcribe(audio_path)

        # Save transcription to a .txt file
        
        transcription_file = os.path.join(transcription_dir, audio_file.replace(".m4a", ".txt"))
        with open(transcription_file, "w") as f:
            f.write(transcription['text'])
        print(f"Transcribed and saved: {transcription_file}")
    except Exception as e:
        print(f"Failed to transcribe {audio_file}: {e}")



Transcribed and saved: /notebooks/path/to/transcriptions/c4181c7e744aaaae5aaf7e234e39a17e.txt
Transcribed and saved: /notebooks/path/to/transcriptions/c813c650315977e7da699c0b2d52799d.txt
Transcribed and saved: /notebooks/path/to/transcriptions/When Cat Introductions Get UGLY.txt
Transcribed and saved: /notebooks/path/to/transcriptions/6e43a9a97ff958b094ee86b59d6fa433.txt
Transcribed and saved: /notebooks/path/to/transcriptions/Can my Cats Get Along? Cat-to-Cat Body Language basics & Introduction Tips.txt
Transcribed and saved: /notebooks/path/to/transcriptions/How To Introduce Your Cat to a New KITTEN!.txt
Transcribed and saved: /notebooks/path/to/transcriptions/How to Introduce Cats.txt
Transcribed and saved: /notebooks/path/to/transcriptions/Are My Cats Playing or Fighting? | Cat Playing vs Cat Aggression.txt
Transcribed and saved: /notebooks/path/to/transcriptions/Cat Introductions Gone Wrong: They Will NOT Work it Out Without You.txt
Transcribed and saved: /notebooks/path/to/trans

# Check if the transcriptions are on the right path

In [ ]:
import os
import glob

# Define the directory for transcription files
transcription_dir = "path/to/transcriptions"

# Check if the directory exists
if not os.path.exists(transcription_dir):
    print(f"The transcription directory {transcription_dir} does not exist.")
else:
    # List transcription files
    transcription_files = glob.glob(os.path.join(transcription_dir, "*.txt"))
    if not transcription_files:
        print(f"No .txt transcription files found in {transcription_dir}.")
    else:
        print(f"Found transcription files: {transcription_files}")


Found transcription files: ["path/to/transcriptions/The Do's & Don'ts of Introducing Cats.txt", 'path/to/transcriptions/c813c650315977e7da699c0b2d52799d.txt', 'path/to/transcriptions/55340c0bd9e5d16b88d7bdc38908eccc.txt', 'path/to/transcriptions/Can my Cats Get Along? Cat-to-Cat Body Language basics & Introduction Tips.txt', 'path/to/transcriptions/When Cat Introductions Get UGLY.txt', 'path/to/transcriptions/c4181c7e744aaaae5aaf7e234e39a17e.txt', 'path/to/transcriptions/fa8c74567ee31f6e4bdfc0abb681f0a4.txt', 'path/to/transcriptions/Brady Bunching: Introducing Two Groups of Cats.txt', 'path/to/transcriptions/Can Cats & Dogs Be Friends? | Jackson Galaxy.txt', 'path/to/transcriptions/b81b63d4601ef135d3641d91fa8e5920.txt', 'path/to/transcriptions/6e43a9a97ff958b094ee86b59d6fa433.txt', 'path/to/transcriptions/Cat Introductions: Good First Impressions are a must!.txt', 'path/to/transcriptions/How to Introduce Cats.txt', 'path/to/transcriptions/This was BAD!! 🙀🤯🫣 #shorts.txt', 'path/to/trans

# Chromadb as Client and creation of Collection

In [ ]:
import chromadb
from sentence_transformers import SentenceTransformer
import os
import glob
import hashlib

# Initialize ChromaDB client and set up the collection
client = chromadb.Client()

# Check if the collection exists; create if not
collection_name = "cat_expert_knowledge"
if collection_name not in [col.name for col in client.list_collections()]:
    collection = client.create_collection(collection_name)
else:
    collection = client.get_collection(collection_name)
    print(f"Using existing collection: {collection_name}")

# Load the embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Directory containing transcription files
transcription_dir = "path/to/transcriptions"

def generate_document_id(text):
    """Generate a unique hash ID for each document based on its text content."""
    return hashlib.md5(text.encode()).hexdigest()

def add_document_to_chromadb(file_path, collection):
    """Adds a document to ChromaDB if it doesn't already exist."""
    # Read file content
    with open(file_path, "r") as f:
        text = f.read()
    
    # Generate a unique ID based on document content
    doc_id = generate_document_id(text)
    
    # Query to check if this document hash already exists
    existing_docs = collection.query(
        query_texts=[doc_id], 
        where={"topic": doc_id}
    )
    if existing_docs['documents']:  # If results found, skip adding
        print(f"Document with ID {doc_id} already exists in ChromaDB. Skipping.")
        return
    
    # Embed the text and add to ChromaDB
    embedding = embedding_model.encode(text).tolist()
    collection.add(
        ids=[doc_id],
        documents=[text],
        metadatas=[{"topic": doc_id, "source": file_path}],
        embeddings=[embedding]
    )
    print(f"Document {doc_id} added to ChromaDB.")

# Process each transcription file
transcription_files = glob.glob(os.path.join(transcription_dir, "*.txt"))
if not transcription_files:
    print("No transcription files found in the specified directory.")

for transcription_file in transcription_files:
    try:
        add_document_to_chromadb(transcription_file, collection)
    except Exception as e:
        print(f"Failed to add {transcription_file} to ChromaDB: {e}")


# Testing and RAG Quering

In [ ]:
from langchain_openai import ChatOpenAI

# Initialize the language model from LangChain
llm = ChatOpenAI()

def query_rag(collection, query_text, embedding_model, llm, n_results=3):
    """
    Perform a Retrieval-Augmented Generation (RAG) query to retrieve relevant documents
    and generate a response using a language model.
    
    Parameters:
    - collection: ChromaDB collection to query.
    - query_text: The question or input text for querying.
    - embedding_model: Sentence embedding model (e.g., SentenceTransformer).
    - llm: Language model from LangChain for generating responses.
    - n_results: Number of relevant documents to retrieve.
    
    Returns:
    - response: Generated answer from the language model based on retrieved documents.
    """
    
    # Embed the query text
    query_embedding = embedding_model.encode(query_text).tolist()
    
    # Perform the search in ChromaDB
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results
    )
    
    # Check if documents are found
    if results['documents']:
        # Prepare the context from retrieved documents
        context_texts = [doc for doc in results['documents'][0]]
        context = "\n".join(context_texts)
        
        # Formulate prompt for the language model with context
        prompt = f"Context:\n{context}\n\nQuestion: {query_text}\nAnswer:"
        
        # Generate response using the language model
        response = llm.invoke(prompt)
        
        return response
    
    else:
        return "No relevant documents found for this query."

# Example Usage
query_text = "How do I stop my cat from scratching furniture?"
response = query_rag(collection, query_text, embedding_model, llm)
print("Generated Response:", response)


Generated Response: content="There are a few ways you can prevent your cat from scratching furniture. One option is to provide them with appropriate scratching posts or pads, so they can satisfy their scratching instinct in a designated area. You can also try using deterrents like double-sided tape or citrus spray on the furniture to discourage them from scratching. Additionally, you can trim your cat's nails regularly or use soft nail caps to prevent damage to furniture. Finally, positive reinforcement techniques, such as rewarding your cat for using the scratching post, can also be effective in redirecting their behavior." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 110, 'prompt_tokens': 3555, 'total_tokens': 3665, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-

Answers not in character.

Trace where is retriving text from.

In [ ]:
from sentence_transformers import SentenceTransformer

# Load the SentenceTransformer model for embeddings
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Sample query to retrieve relevant information
query_text = "How do I stop my cat from scratching furniture?"
query_embedding = embedding_model.encode(query_text).tolist()  # Correct embedding model

# Perform the search in ChromaDB
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=3
)

# Check if results are found
if results['documents']:
    for result in results['documents']:
        print("Retrieved Text:", result)
    for metadata in results['metadatas']:
        print("Metadata:", metadata)
else:
    print("No relevant results found.")


Retrieved Text: [" What's up gang is I Jackson Galaxy your cat daddy and today your cat question answerer that is what I'm here for today hopefully we get questions all the time from folks just like you who sent it into this address right there which you of course can do as well and today we're gonna talk to Kinesha Kinesha what's it all about hi Jackson I'm Kinesha from the Cayman Islands my husband and I have a cat sunshine she's 11 years old and we've always wanted to get another cat to you know keep her company and also rescue from the human society but you know we're a bit worried because she has been the only cat in the home for so long in 11 years and we just wonder whether or not she'll be very receptive to the addition of another cat in the home we'd love your feedback on the best approach to do so you know what can we do to make this a seamless process because we'd really love for it to be a success and to avoid any elimination or scratching or you know any of the behaviors t

In [ ]:
%pip install langchain
%pip install openai
%pip install langchain_community langchain_openai

In [ ]:
import langchain
langchain.__version__

'0.3.7'

In [ ]:
# Initialize context history for conversation continuity
context_history = []

def retrieve_documents(query):
    # This example assumes you have a collection or database to query
    query_embedding = embedding_model.encode(query).tolist()
    result = collection.query(query_embeddings=[query_embedding], n_results=2)
    
    # Log retrieval details if using LangSmith
    langsmith.invoke("document_retrieval", {
        "query": query,
        "retrieved_documents": result['documents']
    })
    
    documents = " ".join(doc if isinstance(doc, str) else " ".join(doc) for doc in result['documents'])
    return documents


# LangSmith Define Functions
We define the functions of Langsmith to start tracing the project.

In [1]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableSequence

def create_qa_chain():
    template = """You are a friendly, expert cat behaviorist inspired by Jackson Galaxy.
                  Use the information below to provide advice in a helpful and engaging way."""
    prompt = ChatPromptTemplate.from_messages([
        ("system", template),
        ("user", "{context}\n\nQuestion: {question}\nAnswer:")
    ])
    
    # Log QA chain creation
    langsmith.invoke("qa_chain_creation", {
        "template": template,
    })
    
    return RunnableSequence(prompt | llm)


In [3]:
import logging

# Simple main agent function without complex tracing
def ask_agent(query):
    # Retrieve relevant context from ChromaDB
    conversation_context = retrieve_documents(query)
    
    # Combine recent exchanges if conversational memory is needed (optional)
    combined_context = conversation_context  # Simplify if no memory needed

    # Generate answer using the QA chain
    answer = get_answer(query)  # Use a simpler answer generation function
    
    # Append the answer to the context history if needed (optional)
    context_history.append(f"Answer: {answer}")  # Only if you want memory
    
    logging.info(f"Query: {query}, Answer: {answer}")
    return answer


In [ ]:
from langchain_openai import ChatOpenAI

# Initialize the language model from LangChain
llm = ChatOpenAI()

def query_rag(collection, query_text, model, llm, n_results=3):
    """
    Perform a Retrieval-Augmented Generation (RAG) query to retrieve relevant documents
    and generate a response using a language model.
    
    Parameters:
    - collection: ChromaDB collection to query.
    - query_text: The question or input text for querying.
    - model: Sentence embedding model (e.g., SentenceTransformer).
    - llm: Language model from LangChain for generating responses.
    - n_results: Number of relevant documents to retrieve.
    
    Returns:
    - response: Generated answer from the language model based on retrieved documents.
    """
    
    # Embed the query text
    query_embedding = model.encode(query_text).tolist()
    
    # Perform the search in ChromaDB
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results
    )
    
    # Check if documents are found
    if results['documents']:
        # Prepare the context from retrieved documents
        context_texts = [doc for doc in results['documents'][0]]
        context = "\n".join(context_texts)
        
        # Formulate prompt for the language model with context
        prompt = f"Context:\n{context}\n\nQuestion: {query_text}\nAnswer:"
        
        # Generate response using the language model
        response = llm.invoke(prompt)
        
        return response
    
    else:
        return "No relevant documents found for this query."

# Example Usage
query_text = "How do I introduce my cat to another cat?"
response = query_rag(collection, query_text, model, llm)
print("Generated Response:", response)

# Example Usage
query_text = "How do I stop my cat from scratching furniture?"
response = query_rag(collection, query_text, model, llm)
print("Generated Response:", response)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated Response: content='To introduce your cat to another cat, it\'s important to follow a gradual and structured process. Start by setting up a base camp for the new cat, where they can have their own space with familiar scents and items. Make sure your existing cat is on a meal schedule and incorporate playtime into their routine. \n\nNext, begin sight swapping, where the cats can get used to each other\'s scents without direct contact. Gradually increase visual access by using a screen door or baby gates to create a safe barrier. \n\nImplement a feeding ritual where the cats can associate each other\'s presence with positive experiences like mealtime. Gradually reduce the distance between the cats during feeding sessions to build comfort and familiarity. \n\nFinally, engage in the "eat play love" exercise where both cats can interact in a positive and playful manner. Monitor their interactions and be prepared to separate them if needed. \n\nOverall, the key is to take things slo

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated Response: content="There are several ways to stop your cat from scratching furniture. One option is to provide your cat with scratching posts or pads made of materials like sisal or cardboard, which can satisfy their natural scratching instincts. You can also try using deterrents like double-sided tape or citrus scents on the furniture to discourage scratching. Additionally, trimming your cat's nails regularly and providing them with plenty of toys and enrichment can help redirect their scratching behavior. It's important to be patient and consistent in training your cat to use appropriate scratching surfaces." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 105, 'prompt_tokens': 3555, 'total_tokens': 3660, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-

# Store Embeddings and Perform Retrieval:
Local Text Library for fast retrival.

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Initialize the embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Example text data
text_data = [
    {"id": "doc1", "content": "Why do cats scratch furniture?"},
    {"id": "doc2", "content": "How to introduce a new cat to your home."},
    # Add more text entries as needed
]

# Generate embeddings for each text entry and store in a dictionary
text_store = {
    entry["id"]: {
        "id": entry["id"],
        "embedding": embedding_model.encode(entry["content"]).tolist(),
        "content": entry["content"]
    }
    for entry in text_data
}

def retrieve_similar_embeddings(query, store, top_k=2):
    """Retrieve top K similar text embeddings from the store."""
    
    # Generate embedding for the query
    query_embedding = embedding_model.encode(query).reshape(1, -1)
    similarities = []

    # Calculate cosine similarity between query and stored embeddings
    for item in store.values():
        score = cosine_similarity(query_embedding, [item["embedding"]]).flatten()[0]
        similarities.append((score, item))

    # Sort by similarity and select top K results
    similarities = sorted(similarities, key=lambda x: x[0], reverse=True)
    return [item for _, item in similarities[:top_k]]

# Example query
query = "How can I stop my cat from scratching furniture?"
results = retrieve_similar_embeddings(query, text_store)

# Output the retrieved results
for result in results:
    print("Retrieved ID:", result["id"])
    print("Content:", result["content"])
